In [22]:
from datetime import datetime, date, time, timedelta
import pandas as pd

In [23]:
n = 1
dist = 963 * n
kml_G = 13.6
price_LG = 7.9
kml_E = 12.0
price_LE = 5.7
capacity = 50

In [24]:
df_fuel = pd.DataFrame(
	{
		'Fuel'				: ['Gasoline', 'Ethanol'],
		'R$/L'				: [price_LG, price_LE],
		'Km/L'				: [kml_G, kml_E]
	}
)

df_fuel['Autonomy (Km)']	= capacity * df_fuel['Km/L']
df_fuel['Km/autonomy']		= dist / df_fuel['Autonomy (Km)']
df_fuel['L/100 Km'] 		= 100 / df_fuel['Km/L']
df_fuel['R$/Km']			= df_fuel['R$/L'] / df_fuel['Km/L']
df_fuel['R$/fill']			= df_fuel['R$/L'] * capacity
df_fuel['R$/100 Km']		= df_fuel['R$/L'] * df_fuel['L/100 Km']
df_fuel['Volume (L)']		= dist / df_fuel['Km/L']
df_fuel['Cost (R$)']		= df_fuel['R$/L'] * df_fuel['Volume (L)']
df_fuel['R$ ratio']			= [price_LG / price_LE, price_LE / price_LG]
df_fuel['Km ratio']			= [kml_G / kml_E, kml_E / kml_G]

In [25]:
one_way = int(dist / n)
avg_speed = 80

In [26]:
df_trip = pd.DataFrame(
	{
		'Distance (Km)'		: [one_way],
		'AVG speed (Km/h)'	: [avg_speed],
		# 'Departure time'	: [datetime.combine(datetime(2022, 4, 28), time(7, 0, 0))]
		'Departure time'	: [datetime.combine(date.today(), time(7, 0, 0))]
	}
)
df_trip['Arrival time'] = df_trip['Departure time'] + timedelta(seconds = int((df_trip['Distance (Km)'] / df_trip['AVG speed (Km/h)']) * 3600))
df_trip['Elapsed time'] = df_trip['Arrival time'] - df_trip['Departure time']
df_time = pd.to_timedelta(df_trip['Elapsed time']).dt.components[['days', 'hours', 'minutes', 'seconds']]
df_time['hours'] = df_time['hours'].add(df_time.pop('days') * 24)

df_trip['Elapsed time hh:mm:ss'] = df_time.astype(str).agg(lambda s: ':'.join(s.str.zfill(2)), axis=1)

In [27]:
print ('Fuel costs')
print ('Tank capacity:', capacity, 'L | Distance:', dist, 'Km')
df_fuel

Fuel costs
Tank capacity: 50 L | Distance: 963 Km


,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio
0,Gasoline,7.9,13.6,680.0,1.416176,7.352941,0.580882,395.0,58.088235,70.808824,559.389706,1.385965,1.133333
1,Ethanol,5.7,12.0,600.0,1.605000,8.333333,0.475000,285.0,47.500000,80.250000,457.425000,0.721519,0.882353


In [28]:
print('Trip calculations')
df_trip

Trip calculations


,Distance (Km),AVG speed (Km/h),Departure time,Arrival time,Elapsed time,Elapsed time hh:mm:ss
0,963,80,2022-04-28 07:00:00,2022-04-28 19:02:15,0 days 12:02:15,12:02:15


In [29]:
print("Fuel's lowest value")
df_fuel[df_fuel['R$/Km'] == df_fuel['R$/Km'].min()]

Fuel's lowest value


,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio
1,Ethanol,5.7,12.0,600.0,1.605,8.333333,0.475,285.0,47.5,80.25,457.425,0.721519,0.882353


In [30]:
print("Fuel's highest value")
df_fuel[df_fuel['R$/Km'] == df_fuel['R$/Km'].max()]

Fuel's highest value


,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio
0,Gasoline,7.9,13.6,680.0,1.416176,7.352941,0.580882,395.0,58.088235,70.808824,559.389706,1.385965,1.133333


In [31]:
print('Trip with the cheapest fuel')
df_cheaper = df_fuel[df_fuel['R$/Km'] == df_fuel['R$/Km'].min()]
df_cheaper.reset_index(inplace=True)
pd.concat([df_cheaper, df_trip], axis=1)

Trip with the cheapest fuel


,index,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio,Distance (Km),AVG speed (Km/h),Departure time,Arrival time,Elapsed time,Elapsed time hh:mm:ss
0,1,Ethanol,5.7,12.0,600.0,1.605,8.333333,0.475,285.0,47.5,80.25,457.425,0.721519,0.882353,963,80,2022-04-28 07:00:00,2022-04-28 19:02:15,0 days 12:02:15,12:02:15


In [35]:
print('Trip with the most expensive fuel')
df_cheaper = df_fuel[df_fuel['R$/Km'] == df_fuel['R$/Km'].max()]
df_cheaper.reset_index(inplace=True)
pd.concat([df_cheaper, df_trip], axis=1)

Trip with the most expensive fuel


,index,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio,Distance (Km),AVG speed (Km/h),Departure time,Arrival time,Elapsed time,Elapsed time hh:mm:ss
0,0,Gasoline,7.9,13.6,680.0,1.416176,7.352941,0.580882,395.0,58.088235,70.808824,559.389706,1.385965,1.133333,963,80,2022-04-28 07:00:00,2022-04-28 19:02:15,0 days 12:02:15,12:02:15


In [32]:
print("Trip with the highest autonomy fuel")
df_most_autonomy = df_fuel[df_fuel['Autonomy (Km)'] == df_fuel['Autonomy (Km)'].max()]
df_most_autonomy.reset_index(inplace=True)
pd.concat([df_most_autonomy, df_trip], axis=1)

Trip with the highest autonomy fuel


,index,Fuel,R$/L,Km/L,Autonomy (Km),Km/autonomy,L/100 Km,R$/Km,R$/fill,R$/100 Km,Volume (L),Cost (R$),R$ ratio,Km ratio,Distance (Km),AVG speed (Km/h),Departure time,Arrival time,Elapsed time,Elapsed time hh:mm:ss
0,0,Gasoline,7.9,13.6,680.0,1.416176,7.352941,0.580882,395.0,58.088235,70.808824,559.389706,1.385965,1.133333,963,80,2022-04-28 07:00:00,2022-04-28 19:02:15,0 days 12:02:15,12:02:15
